In [ ]:
# default_exp docstring

# Exporting Docstrings

> Converts `docment` docstrings to Numpy styled

In [ ]:
#hide
from nbdev.showdoc import *
from fastcore.test import *

The goal of this module is to take code that looks like the following:

In [ ]:
def addition(
    a:int, # The first number to add
    b:int=2, # The second number to add
) -> int: # The sum of a and b
    "Adds two numbers together"
    return a+b

And convert it to be the following:

In [ ]:
def addition(a,b) -> int:
    """Adds two numbers together
    
    Parameters
    ---------
    a : int
        The first number to add
    b : int
        The second number to add
        
    Returns
    -------
    int
        The sum of a and b
    """
    return a + b

In [ ]:
# TESTING CODE
import nbdev.export as exp
nb = exp.read_nb('99_test.ipynb')
default = exp.find_default_export(nb['cells'])
mod = exp.get_nbdev_module()
exports = [exp.is_export(c, default) for c in nb['cells']]
cells = [(i,c,e) for i,(c,e) in enumerate(zip(nb['cells'],exports)) if e is not None]
flag_lines, code_lines = exp.split_flags_and_code(cells[0][1])
code_lines = exp._deal_import(code_lines, 'docstring.py')

In [ ]:
# TESTING CODE
code_lines

['def addition(',
 '    a:int, # The first number to add',
 '    b:int, # The second number to add',
 ') -> int: # The sum of a and b',
 '    "Adds two numbers together"',
 '    return a+b']

In [ ]:
#export
import inspect, ast, astunparse
from __future__ import annotations
import fastcore.docments as dments
from collections import OrderedDict

from fastcore.xtras import risinstance

Below we have an example string repsentation of the above docments style:

In [ ]:
source = '''def addition(
    a:(int, float), # The first number to add
    # The second number to add
    b:int = 2,
) -> (int,float): # The sum of a and b
    "Adds two numbers together"
    return a+b'''

In [ ]:
#export
def get_annotations(
    source:str # Source code of function or class
):
    "Extracts the type annotations from source code"
    parse = ast.parse(source)
    arg_annos = []
    for i,anno in enumerate(parse.body[0].args.args):
        if anno.annotation is not None:
            arg_annos.append(astunparse.unparse(anno.annotation).strip('\n'))
        else:
            arg_annos.append(anno.annotation)
        parse.body[0].args.args[i].annotation = None
    if parse.body[0].returns is not None:
        ret_anno = astunparse.unparse(parse.body[0].returns).strip('\n')
    else:
        ret_anno = None
    return arg_annos, ret_anno

In [ ]:
test_eq(get_annotations(source), (['(int, float)', 'int'], '(int, float)'))

In [ ]:
#export
def _get_leading(o):
    return len(o) - len(o.lstrip(o[0])), o[0]

In [ ]:
test_eq(_get_leading('  Hello my name is Zach'), (2, ' '))

In [ ]:
def testme(anno):
    print(anno or 'any')

In [ ]:
testme(None)

any


In [ ]:
def o():
    param_string = f'\n{_get_whitespace()}Parameters\n{_get_whitespace()}----------\n'
    def _inner(param, anno):
        param_string = ''
        if param not in ['return', 'self', 'cls']:
            param_string += f'{_get_whitespace()}{param}'
            param_string += f' : {anno or "any"}\n'
            print(param)
            if docs[param] is not None:
                param_string += f'{whitespace_char * (num_whitespace+2)}{docs[param]}\n'
        return param_string
    
    o = apply(_inner, docks.values(), annos[0])
    param_string += '\n'.join(o)
    if param_string != f'\n{_get_whitespace()}Parameters\n{_get_whitespace()}----------\n':
        docstring += param_string

In [ ]:
ComplexityVisitor.from_code(inspect.getsource(o)).functions

[Function(name='o', lineno=1, col_offset=0, endline=16, is_method=False, classname=None, closures=[Function(name='_inner', lineno=3, col_offset=4, endline=11, is_method=False, classname=None, closures=[], complexity=4)], complexity=2)]

In [ ]:
_quotes = ("'", '"')

In [ ]:
orig_docstring = astunparse.unparse(ast.parse(source).body[0].body[0]).lstrip(' ').replace(_quotes[0],'').replace(_quotes[1],'')
orig_docstring = orig_docstring.split('\\n')

In [ ]:
o(orig_docstring, ' ')

'\n \nAdds two numbers together\n'

In [ ]:
o(orig_docstring, ' ')

'\n \nAdds two numbers together\n'

In [ ]:
ComplexityVisitor.from_code(inspect.getsource(o)).functions

[Function(name='o', lineno=1, col_offset=0, endline=11, is_method=False, classname=None, closures=[], complexity=6)]

In [ ]:
ComplexityVisitor.from_code(inspect.getsource(o)).functions

[Function(name='o', lineno=1, col_offset=0, endline=14, is_method=False, classname=None, closures=[], complexity=6)]

In [ ]:
def apply(func, x, *args, **kwargs):
    "Apply `func` recursively to `x`, passing on args"
    if is_listy(x): return type(x)([apply(func, o, *args, **kwargs) for o in x])
    if isinstance(x,dict):  return {k: apply(func, v, *args, **kwargs) for k,v in x.items()}
    res = func(x, *args, **kwargs)
    return res

In [ ]:
p_source = ast.parse(source)

In [ ]:
p_source.body[0].args.args

[<_ast.arg at 0x7f27ed9e3438>, <_ast.arg at 0x7f27ed9e3e10>]

In [ ]:
#export
from fastcore.xtras import is_listy

In [ ]:
ast.parse("""@typedispatch
def o(): 
    for i in range(len(parsed_source.body[0].args.args)):
        parsed_source.body[0].args.args[i].annotation = None""").body[0].decorator_list

In [ ]:
ComplexityVisitor.from_code("""@typedispatch
def o(): 
    for i in range(len(parsed_source.body[0].args.args)):
        parsed_source.body[0].args.args[i].annotation = None""").functions

[Function(name='o', lineno=1, col_offset=0, endline=4, is_method=False, classname=None, closures=[], complexity=2)]

In [ ]:
%%timeit
p_source = ast.parse(source)
_ = apply(lambda x: setattr(x, 'annotation', None), p_source.body[0].args.args)

33.5 µs ± 2.87 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [ ]:
%%timeit
p_source = ast.parse(source)
for i in range(len(p_source.body[0].args.args)):
        p_source.body[0].args.args[i].annotation = None

22.1 µs ± 1.26 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [ ]:
#export
def reformat_function(
    source:str, # Source code
):
    "Takes messy source code and refactors it into a readable PEP-8 standard style"
    # Read in docments, parse source code, generate annotations
    docs = dments.docments(source)
    parsed_source = ast.parse(source)
    annos = get_annotations(source)
    
    # Set all arg annotations to None
    _ = apply(lambda x: setattr(x, 'annotation', None), parsed_source.body[0].args.args)
        
    parsed_source.body[0].returns = None
    body = parsed_source.body[0].body
    unparsed_source = astunparse.unparse(parsed_source).lstrip('\n').split('\n')
    has_decorator = len(parsed_source.body[0].decorator_list) > 0
        
    # Extract function definition
    function_definition = '\n'.join(unparsed_source[:2]) if has_decorator else unparsed_source[0]

    # Check if we have a docstring and extract function innards
    def _extract_innards(is_str:bool):
        i = 2 if is_str else 1
        return '\n'.join(unparsed_source[i+1:]) if has_decorator else '\n'.join(unparsed_source[i:])
    
    function_innards = _extract_innards(isinstance(body[0].value, ast.Str))
            
    def _get_whitespace(): return whitespace_char*num_whitespace
    
    if unparsed_source[2] != '':
        num_whitespace, whitespace_char = _get_leading(unparsed_source[2])
    else:
        if len(unparsed_source) < 4:
            num_whitespace, whitespace_char = _get_leading(unparsed_source[1])
        else:
            num_whitespace, whitespace_char = _get_leading(unparsed_source[3])
        
    docstring = f'\n{_get_whitespace()}"""'
    
    if isinstance(body[0].value, ast.Str):
        _quotes = ("'", '"')
        orig_docstring = astunparse.unparse(body[0]).lstrip(whitespace_char).replace(_quotes[0],'').replace(_quotes[1],'')
        orig_docstring = orig_docstring.split('\\n')
        def _inner(line, whitespace_char):
            diff = len(line) - len(line.lstrip())
            whitespace = whitespace_char * diff if diff > 0 else _get_whitespace()
            return f'\n{whitespace}{line}'

        o = apply(_inner, orig_docstring, whitespace_char=whitespace_char)
        o[0] = orig_docstring[0].lstrip()
        docstring +=  '\n'.join(o)
        
    param_string = f'\n{_get_whitespace()}Parameters\n{_get_whitespace()}----------\n'
    if len(docs.keys()) >= 1:
        param_string = f'\n{_get_whitespace()}Parameters\n{_get_whitespace()}----------\n'
        for i, param in enumerate(docs.keys()):
            if param != "return" and param != "self" and param != "cls":
                param_string += f'{_get_whitespace()}{param}'
                if annos[0][i] is not None:
                    param_string += f' : {annos[0][i]}'
                else:
                    param_string += f' : any'
                param_string += '\n'
                if docs[param] is not None:
                    param_string += f'{whitespace_char * (num_whitespace+2)}{docs[param]}\n'
    if param_string != f'\n{_get_whitespace()}Parameters\n{_get_whitespace()}----------\n':
        docstring += param_string
            
    if (annos[-1] != inspect._empty) and ('return' in docs.keys()):
        docstring += f'\n{_get_whitespace()}Returns\n'
        docstring += f'{_get_whitespace()}-------\n'
        docstring += f'{_get_whitespace()}{annos[1]}\n'
        docstring += f'{whitespace_char * (num_whitespace+2)}{docs["return"]}\n'
    docstring += f'{_get_whitespace()}"""\n'
    return f'{function_definition}{docstring}{function_innards}'

In [ ]:
source = """@delegates()
def addition(
    a:(int, float), # The first number to add
    # The second number to add
    b:int = 2,
) -> (int,float): # The sum of a and b
    "Adds two numbers together"
    def _inner(): return a+b
    return _inner()"""

In [ ]:
print(source)

@delegates()
def addition(
    a:(int, float), # The first number to add
    # The second number to add
    b:int = 2,
) -> (int,float): # The sum of a and b
    "Adds two numbers together"
    def _inner(): return a+b
    return _inner()


In [ ]:
print(reformat_function(source))

@delegates()
def addition(a, b=2):
    """Adds two numbers together

    Parameters
    ----------
    a : (int, float)
      The first number to add
    b : int
      The second number to add

    Returns
    -------
    (int, float)
      The sum of a and b
    """

    def _inner():
        return (a + b)
    return _inner()



In [ ]:
# export
def reformat_class(
    source:str, # Source code of a full class
    recursion_level:int = 1, # Depth of recursion
):
    "Takes messy class code and refactors it into a readable PEP-8 standard style"
    whitespace_char = None
    def _format_spacing(code, num_leading):
        code = [c for c in code if len(c) > 0]
        def _inner(c, num_leading):
            curr_leading = len(c) - len(c.lstrip())
            return f'{c[0] * (curr_leading - num_leading)}{c.lstrip()}'
        return apply(_inner, code, num_leading=num_leading)
    # Parse source code and get body
    parsed_source = ast.parse(source)
    body = parsed_source.body[0].body
    new_source = ''
    
    unparsed_source = astunparse.unparse(parsed_source).lstrip('\n').split('\n')
    
    # Add function definition
    new_source += unparsed_source[0]
    
    def _get_whitespace(): return whitespace_char * num_whitespace
    
    num_whitespace, whitespace_char = _get_leading(unparsed_source[2])
    
    docstring = f'\n{_get_whitespace()}"""'
    docstring_len, diff = 0,2
    
    new_nodes = [unparsed_source[0]]
    
    for i, node in enumerate(body):
        if risinstance((ast.ClassDef, ast.FunctionDef), node):
            beginning_lineno = node.lineno
            split_code = source.split('\n')
            if i < (len(body) - 1):
                ending_lineno = body[i+1].lineno
                code = split_code[beginning_lineno-1:ending_lineno-1]
                num_leading = len(code[0]) - len(code[0].lstrip())
                if isinstance(node, ast.ClassDef):
                    for i,c in enumerate(code): code[i] = code[i][num_leading:]
                    new_nodes.append(reformat_class('\n'.join(code), recursion_level+1))
                else:
                    if whitespace_char is None:
                        whitespace_char = code[i][0]
                    code = _format_spacing(code, num_leading)
                    new_nodes.append(reformat_function('\n'.join(code)))
            else:
                code = split_code[beginning_lineno-1:]
                if whitespace_char is None:
                    whitespace_char = code[i][0]
                num_leading = len(code[0]) - len(code[0].lstrip())
                code = _format_spacing(code, num_leading)
                new_nodes.append(reformat_function('\n'.join(code)))
        else:
            if isinstance(body[0].value, ast.Str) and i == 0:
                _quotes = ("'", '"')
                orig_docstring = astunparse.unparse(body[0]).lstrip(whitespace_char).replace(_quotes[0],'').replace(_quotes[1],'')
                orig_docstring = orig_docstring.split('\\n')
                def _inner(line, whitespace_char):
                    diff = len(line) - len(line.lstrip())
                    whitespace = whitespace_char * diff if diff > 0 else _get_whitespace()
                    return f'\n{whitespace}{line}'
                
                o = apply(_inner, orig_docstring, whitespace_char=whitespace_char)
                o[0] = orig_docstring[0].lstrip()
                docstring +=  '\n'.join(o)
                docstring += f'\n{_get_whitespace()}"""'
                full_string = docstring.split('\n')
                new_string = ''
                
                if len(full_string) == 4:
                    new_string = apply(lambda x: x.lstrip(), full_string)
                    new_string = ''.join(new_string)
                else:
                    new_string = '\n'.join(full_string)
                docstring_len = len(new_string.split('\n'))
                new_nodes.append(new_string)
            else:
                new_nodes.append(f'{astunparse.unparse(node).strip()}')
    formatted_source = []
    num_chars = 4
    if recursion_level > 1: 
        num_chars += (2*(recursion_level-1)) - 2
        
    formatted_source.append(new_nodes[0])
    line = new_nodes[1]
    if not len(line.lstrip()) < len(line):
        line = line.split('\n')
        line = apply(lambda x: f'{whitespace_char * num_chars}{x}', line)
        line = '\n'.join(line)
    formatted_source.append(line.lstrip('\n'))
            
    for i,line in enumerate(new_nodes[2:]):
        l = line.split('\n')
        for i,o in enumerate(l): 
            l[i] = f'{whitespace_char * num_chars}{o}'
        line = '\n'.join(l)
        formatted_source.append(line)
    return '\n'.join(formatted_source)

In [ ]:
source = '''class Arithmetic:
    "A class that can perform basic arithmetic on ops"
    _o = 2
    _b = 5
    
    _c = 3
    
    class A:
        def __init__(
          self, 
          o:int # An integer
        ):
            self.o = o
    
    def __init__(
        self,
        a:int, # The first number to use
        b:(int, float), # The second number to use
    ):
        self.a = a
        self.b = b
        
    @typedispatch
    def add(
        self
    ) -> (int,float): # Sum of a and b
        "Adds self.a and self.b"
        return self.a + self.b'''

In [ ]:
print(reformat_class(source))

class Arithmetic():
    """A class that can perform basic arithmetic on ops"""
    _o = 2
    _b = 5
    _c = 3
    class A():
        def __init__(self, o):
            """
            Parameters
            ----------
            o : int
              An integer
            """
            self.o = o
        
    def __init__(self, a, b):
        """
        Parameters
        ----------
        a : int
          The first number to use
        b : (int, float)
          The second number to use
        """
        self.a = a
        self.b = b
    
    @typedispatch
    def add(self):
        """Adds self.a and self.b
    
        Returns
        -------
        (int, float)
          Sum of a and b
        """
        return (self.a + self.b)
    


In [ ]:
class ExampleClass(object):
    """The summary line for a class docstring should fit on one line.

    If the class has public attributes, they may be documented here
    in an ``Attributes`` section and follow the same formatting as a
    function's ``Args`` section. Alternatively, attributes may be documented
    inline with the attribute's declaration (see __init__ method below).

    Properties created with the ``@property`` decorator should be documented
    in the property's getter method.

    Attributes
    ----------
    attr1 : str
        Description of `attr1`.
    attr2 : :obj:`int`, optional
        Description of `attr2`.

    """

    def __init__(self, param1, param2, param3):
        """Example of docstring on the __init__ method.

        The __init__ method may be documented in either the class level
        docstring, or as a docstring on the __init__ method itself.

        Either form is acceptable, but the two should not be mixed. Choose one
        convention to document the __init__ method and be consistent with it.

        Note
        ----
        Do not include the `self` parameter in the ``Parameters`` section.

        Parameters
        ----------
        param1 : str
            Description of `param1`.
        param2 : :obj:`list` of :obj:`str`
            Description of `param2`. Multiple
            lines are supported.
        param3 : :obj:`int`, optional
            Description of `param3`.

        """
        self.attr1 = param1
        self.attr2 = param2
        self.attr3 = param3  #: Doc comment *inline* with attribute

        #: list of str: Doc comment *before* attribute, with type specified
        self.attr4 = ["attr4"]

        self.attr5 = None
        """str: Docstring *after* attribute, with type specified."""

    @property
    def readonly_property(self):
        """str: Properties should be documented in their getter method."""
        return "readonly_property"

    @property
    def readwrite_property(self):
        """:obj:`list` of :obj:`str`: Properties with both a getter and setter
        should only be documented in their getter method.

        If the setter method contains notable behavior, it should be
        mentioned here.
        """
        return ["readwrite_property"]

    @readwrite_property.setter
    def readwrite_property(self, value):
        value

    def example_method(self, param1, param2):
        """Class methods are similar to regular functions.

        Note
        ----
        Do not include the `self` parameter in the ``Parameters`` section.

        Parameters
        ----------
        param1
            The first parameter.
        param2
            The second parameter.

        Returns
        -------
        bool
            True if successful, False otherwise.

        """
        return True

    def __special__(self):
        """By default special members with docstrings are not included.

        Special members are any methods or attributes that start with and
        end with a double underscore. Any special member with a docstring
        will be included in the output, if
        ``napoleon_include_special_with_doc`` is set to True.

        This behavior can be enabled by changing the following setting in
        Sphinx's conf.py::

            napoleon_include_special_with_doc = True

        """
        pass

    def __special_without_docstring__(self):
        pass

    def _private(self):
        """By default private members are not included.

        Private members are any methods or attributes that start with an
        underscore and are *not* special. By default they are not included
        in the output.

        This behavior can be changed such that private members *are* included
        by changing the following setting in Sphinx's conf.py::

            napoleon_include_private_with_doc = True

        """
        pass

    def _private_without_docstring(self):
        pass